In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import os
from pathlib import Path
from urllib.parse import urljoin, urlparse
import time
import logging
from typing import List, Dict

In [ ]:
class TikiScraperWithMetadata:
    def __init__(self, base_dir="tiki_dataset"):
        self.base_dir = Path(base_dir)
        self.base_dir.mkdir(exist_ok=True)

        # Setup requests session
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'vi-VN,vi;q=0.9,en;q=0.8',
            'Accept-Encoding': 'gzip, deflate, br',
            'DNT': '1',
            'Connection': 'keep-alive'
        })

        # Setup logging
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger(__name__)

        # Danh sách metadata cho tất cả sản phẩm
        self.products_metadata = []

    def extract_product_info_from_url(self, product_url: str) -> Dict:
        """
        Trích xuất thông tin sản phẩm từ URL

        Logic:
        - URL: https://tiki.vn/product-p186389538.html?spid=186389559
        - Product ID: 186389538 (từ p{ID}.html)
        - SPID: 186389559 (từ spid parameter)
        """
        product_info = {
            "id": "",
            "spid": "",
            "short_url": product_url
        }

        # Extract Product ID từ pattern p{ID}.html
        match = re.search(r'p(\d+)\.html', product_url)
        if match:
            product_info["id"] = match.group(1)

        # Extract SPID từ URL parameters
        match = re.search(r'spid=(\d+)', product_url)
        if match:
            product_info["spid"] = match.group(1)

        # Nếu không có SPID, dùng Product ID
        if not product_info["spid"]:
            product_info["spid"] = product_info["id"]

        self.logger.debug(f"Extracted: ID={product_info['id']}, SPID={product_info['spid']}")
        return product_info

    def scrape_product_details(self, product_url: str) -> Dict:
        """
        Scrape chi tiết sản phẩm: tên, ảnh, v.v.

        Returns:
        {
            "id": "186389538",
            "name": "đồ bộ mặc nhà",
            "short_url": "https://tiki.vn/product-p186389538.html?spid=186389559",
            "annotations": "",
            "images": ["186389538_01.png", "186389538_02.png", ...]
        }
        """
        self.logger.info(f" Scraping product: {product_url}")

        # Extract basic info từ URL
        product_info = self.extract_product_info_from_url(product_url)

        # Khởi tạo structure
        product_data = {
            "id": product_info["id"],
            "name": "",
            "short_url": product_info["short_url"],
            "annotations": "",
            "images": []
        }

        try:
            response = self.session.get(product_url, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')

            # EXTRACT TÊN SẢN PHẨM
            product_name = self.extract_product_name(soup)
            product_data["name"] = product_name

            # EXTRACT ẢNH SẢN PHẨM
            image_urls = self.extract_product_images(soup, product_url)

            # TẠO THƯ MỤC SẢN PHẨM
            product_dir = self.create_product_directory(product_info["id"])

            # DOWNLOAD VÀ LƯU ẢNH
            saved_images = self.download_and_save_images(
                image_urls,
                product_dir,
                product_info["id"]
            )

            product_data["images"] = saved_images

            self.logger.info(f"✅ Scraped product {product_info['id']}: {len(saved_images)} images")

        except Exception as e:
            self.logger.error(f"❌ Error scraping {product_url}: {e}")

        return product_data

    def extract_product_name(self, soup: BeautifulSoup) -> str:
        """
        Extract tên sản phẩm từ HTML

        Logic: Tìm theo thứ tự ưu tiên
        1. h1 với data-view-id cụ thể
        2. h1.title
        3. .product-name
        4. h1 đầu tiên
        """
        name_selectors = [
            'h1[data-view-id="pdp_details_view_product_title"]',
            'h1.title',
            '.product-name h1',
            '.product-title',
            'h1'
        ]

        for selector in name_selectors:
            element = soup.select_one(selector)
            if element:
                name = element.get_text().strip()
                if name:
                    self.logger.debug(f"Found name via {selector}: {name[:50]}...")
                    return name

        self.logger.warning("Could not extract product name")
        return "Unknown Product"

    def extract_product_images(self, soup: BeautifulSoup, base_url: str) -> List[str]:
        """
        Extract tất cả ảnh sản phẩm từ HTML

        Logic:
        1. Tìm ảnh chính trong gallery
        2. Tìm ảnh trong slider/carousel
        3. Tìm tất cả ảnh có domain tikicdn
        4. Upgrade thành ảnh chất lượng cao
        """
        image_urls = set()

        # Method 1: Ảnh chính trong product gallery
        main_img_selectors = [
            'img[data-view-id="pdp_details_view_product_image"]',
            '.product-image img',
            '.gallery-image img',
            '.image-gallery img'
        ]

        for selector in main_img_selectors:
            images = soup.select(selector)
            for img in images:
                src = img.get('src') or img.get('data-src') or img.get('data-original')
                if src:
                    image_urls.add(src)

        # Method 2: Ảnh trong slider/carousel
        slider_selectors = [
            '.slider img',
            '.carousel img',
            '.product-gallery img',
            '.thumbnail img'
        ]

        for selector in slider_selectors:
            images = soup.select(selector)
            for img in images:
                src = img.get('src') or img.get('data-src')
                if src:
                    image_urls.add(src)

        # Method 3: Tất cả ảnh có domain Tiki
        all_images = soup.find_all('img')
        for img in all_images:
            src = img.get('src') or img.get('data-src')
            if src and ('tikicdn.com' in src or 'tiki.vn' in src):
                image_urls.add(src)

        # Xử lý URLs
        processed_urls = []
        for url in image_urls:
            # Convert to full URL
            if url.startswith('//'):
                url = 'https:' + url
            elif not url.startswith('http'):
                url = urljoin(base_url, url)

            # Upgrade to high quality
            if 'cache.tikicdn.com' in url:
                # Thay đổi kích thước thành 1200x1200 hoặc gỡ bỏ để lấy ảnh gốc
                url = re.sub(r'/cache/\d+x\d+/', '/cache/1200x1200/', url)

            # Lọc ảnh có kích thước hợp lý (loại bỏ icon, logo nhỏ)
            if not any(x in url.lower() for x in ['icon', 'logo', 'badge', 'thumb']):
                processed_urls.append(url)

        # Loại bỏ duplicate và giới hạn số lượng
        unique_urls = list(dict.fromkeys(processed_urls))[:10]  # Max 10 ảnh

        self.logger.debug(f"Found {len(unique_urls)} unique images")
        return unique_urls

    def create_product_directory(self, product_id: str) -> Path:
        """
        Tạo thư mục cho sản phẩm
        Structure: base_dir/product_id/
        """
        product_dir = self.base_dir / str(product_id)
        product_dir.mkdir(exist_ok=True)

        self.logger.debug(f"Created directory: {product_dir}")
        return product_dir

    def download_and_save_images(self, image_urls: List[str],
                                product_dir: Path, product_id: str) -> List[str]:
        """
        Download và lưu ảnh với tên có thứ tự

        Returns: List tên file đã lưu
        Format: {product_id}_01.png, {product_id}_02.png, ...
        """
        saved_images = []

        for i, image_url in enumerate(image_urls, 1):
            try:
                # Tạo tên file với format chuẩn
                filename = f"{product_id}_{i:02d}.png"
                filepath = product_dir / filename

                # Skip nếu file đã tồn tại
                if filepath.exists():
                    self.logger.debug(f"File exists, skipping: {filename}")
                    saved_images.append(filename)
                    continue

                # Download image
                self.logger.info(f"⬇️ Downloading: {filename}")

                response = self.session.get(image_url, stream=True, timeout=30)
                response.raise_for_status()

                # Kiểm tra content type
                content_type = response.headers.get('content-type', '')
                if not content_type.startswith('image/'):
                    self.logger.warning(f"Not an image: {image_url}")
                    continue

                # Lưu file
                with open(filepath, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)

                # Kiểm tra file size (loại bỏ ảnh quá nhỏ)
                if filepath.stat().st_size < 8000:  # < 1KB
                    filepath.unlink()  # Xóa file
                    self.logger.warning(f"Image too small, deleted: {filename}")
                    continue

                saved_images.append(filename)
                self.logger.info(f"✅ Saved: {filename}")

                # Delay giữa các download
                time.sleep(0.5)

            except Exception as e:
                self.logger.error(f"❌ Error downloading {image_url}: {e}")
                continue

        return saved_images

    def scrape_category_products(self, category_url: str, max_pages: int = 5) -> List[str]:
        """
        Lấy danh sách URL sản phẩm từ category

        Returns: List các product URLs
        """
        self.logger.info(f"🔍 Scraping category: {category_url}")

        all_product_urls = []

        for page in range(1, max_pages + 1):
            page_url = f"{category_url}?page={page}"
            self.logger.info(f" Page {page}: {page_url}")

            try:
                response = self.session.get(page_url, timeout=10)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')

                # Tìm product links
                product_links = []

                # Method 1: Link có data-view-id
                links = soup.find_all('a', {'data-view-id': 'pdp_main'})
                for link in links:
                    href = link.get('href')
                    if href:
                        product_links.append(href)

                # Method 2: Link có pattern p{ID}.html
                if not product_links:
                    links = soup.find_all('a', href=re.compile(r'.*p\d+\.html'))
                    for link in links:
                        href = link.get('href')
                        if href:
                            product_links.append(href)

                # Convert to full URLs
                for href in product_links:
                    if href.startswith('/'):
                        full_url = f"https://tiki.vn{href}"
                    else:
                        full_url = href

                    if full_url not in all_product_urls:
                        all_product_urls.append(full_url)

                self.logger.info(f"Found {len(product_links)} products on page {page}")

                # Delay giữa các trang
                time.sleep(1)

            except Exception as e:
                self.logger.error(f"Error scraping page {page}: {e}")
                continue

        self.logger.info(f"Total products found: {len(all_product_urls)}")
        return all_product_urls

    def scrape_full_category(self, category_url: str, max_pages: int = 5,
                           max_products: int = 50) -> str:
        """
        Scrape toàn bộ category và tạo JSON metadata

        Returns: Path to JSON file
        """
        self.logger.info(f" Starting full category scrape: {category_url}")

        # Reset metadata
        self.products_metadata = []

        # Lấy danh sách sản phẩm
        product_urls = self.scrape_category_products(category_url, max_pages)

        # Giới hạn số sản phẩm
        if len(product_urls) > max_products:
            product_urls = product_urls[:max_products]
            self.logger.info(f"Limited to {max_products} products")

        # Scrape từng sản phẩm
        for i, product_url in enumerate(product_urls, 1):
            self.logger.info(f"🔄 Processing product {i}/{len(product_urls)}")

            product_data = self.scrape_product_details(product_url)

            # Chỉ thêm vào metadata nếu có ảnh
            if product_data["images"]:
                self.products_metadata.append(product_data)

            # Delay giữa các sản phẩm
            time.sleep(1)

        # Lưu JSON metadata
        json_file = self.save_metadata_json()

        self.logger.info(f" Completed! Total products: {len(self.products_metadata)}")
        self.logger.info(f" JSON metadata saved: {json_file}")

        return json_file

    def save_metadata_json(self) -> str:
        """
        Lưu metadata thành file JSON (giống như file của anh bạn)
        """
        json_file = self.base_dir / "products_metadata.json"

        with open(json_file, 'w', encoding='utf-8') as f:
            json.dump(self.products_metadata, f, ensure_ascii=False, indent=4)

        # Tạo summary
        total_products = len(self.products_metadata)
        total_images = sum(len(p["images"]) for p in self.products_metadata)

        summary = {
            "total_products": total_products,
            "total_images": total_images,
            "products": self.products_metadata
        }

        summary_file = self.base_dir / "summary.json"
        with open(summary_file, 'w', encoding='utf-8') as f:
            json.dump(summary, f, ensure_ascii=False, indent=4)

        return str(json_file)

# DEMO: Sử dụng giống như script của anh bạn
def main():
    scraper = TikiScraperWithMetadata("tiki_fashion_dataset")

    # Scrape thời trang nữ (giống data của anh bạn)
    category_url = "https://tiki.vn/nha-sach-tiki/c8322"

    json_file = scraper.scrape_full_category(
        category_url=category_url,
        max_pages=3,
        max_products=25
    )

    print(f" Hoàn thành! JSON file: {json_file}")

    # In thống kê
    print(f"  Thống kê:")
    print(f"   - Tổng sản phẩm: {len(scraper.products_metadata)}")
    print(f"   - Tổng ảnh: {sum(len(p['images']) for p in scraper.products_metadata)}")

if __name__ == "__main__":
    main()

In [ ]:
!rm -r /content/tiki_books_dataset

rm: cannot remove '/content/tiki_books_dataset': No such file or directory


In [ ]:
!rm -r /content/tiki_fashion_dataset/